In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


In [1]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, bidirectional=False):
        super(LSTMModel, self).__init__()
        
        # Define LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=bidirectional)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size * (2 if bidirectional else 1), output_size)

    def forward(self, x):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.lstm.num_layers * (2 if self.lstm.bidirectional else 1), x.size(0), self.lstm.hidden_size).to(x.device)
        c0 = torch.zeros(self.lstm.num_layers * (2 if self.lstm.bidirectional else 1), x.size(0), self.lstm.hidden_size).to(x.device)
        
        # LSTM output
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, sequence_length, hidden_size)
        
        # Get the last time step output
        out = out[:, -1, :]  # Take the output of the last time step for classification

        # Pass through the fully connected layer
        out = self.fc(out)
        return out


NameError: name 'nn' is not defined

In [3]:
import numpy as np

# Generate synthetic data
sequence_length = 10  # Length of each sequence
input_size = 5       # Number of features in each step of the sequence
num_samples = 1000   # Number of sequences

# Randomly generate sequences and labels
X = np.random.rand(num_samples, sequence_length, input_size).astype(np.float32)
y = np.random.randint(0, 2, size=(num_samples,)).astype(np.float32)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X)
y_tensor = torch.tensor(y, dtype=torch.long)

# Create a DataLoader for batching
dataset = TensorDataset(X_tensor, y_tensor)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [4]:
# Hyperparameters
input_size = 5
hidden_size = 64
output_size = 2  # Binary classification (0 or 1)
num_layers = 1
learning_rate = 0.001

# Model, loss, optimizer
model = LSTMModel(input_size, hidden_size, output_size, num_layers=num_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [5]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for sequences, labels in train_loader:
        sequences, labels = sequences.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")


Epoch [1/10], Loss: 0.6957
Epoch [2/10], Loss: 0.6939
Epoch [3/10], Loss: 0.6938
Epoch [4/10], Loss: 0.6932
Epoch [5/10], Loss: 0.6930
Epoch [6/10], Loss: 0.6927
Epoch [7/10], Loss: 0.6933
Epoch [8/10], Loss: 0.6917
Epoch [9/10], Loss: 0.6921
Epoch [10/10], Loss: 0.6919


In [6]:
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for sequences, labels in loader:
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Test on training set (or use a separate test set if available)
train_accuracy = evaluate(model, train_loader, device)
print(f'Training Accuracy: {train_accuracy:.2f}%')


Training Accuracy: 55.50%
